In [1]:
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db_local = 'telecomm_csi.db'
path_to_db_platform = '/datasets/telecomm_csi.db'
path_to_db = None

if os.path.exists(path_to_db_local):
    path_to_db = path_to_db_local
elif os.path.exists(path_to_db_platform):
    path_to_db = path_to_db_platform
else:
    raise Exception('Файл с базой данных SQLite не найден!')

if path_to_db:
    engine = create_engine(f'sqlite:///{path_to_db}', echo=False)

In [3]:
query = """
SELECT *
    FROM user;
"""

In [4]:
query = """
SELECT user_id, lt_day, 
CASE
    WHEN lt_day <= 365 THEN 'new'
    WHEN lt_day > 365 THEN 'old'
END AS is_new, 
age, gender_segment, os_name, cpe_type_name, loc.country, loc.city, a.title AS age_segment, 
tr.title AS traffic_segment, lt.title AS lifetime_segment, nps_score,
CASE
    WHEN nps_score > 0 AND nps_score <= 6 THEN 'detractors'
    WHEN nps_score > 6 AND nps_score <= 8 THEN 'passives'
    WHEN nps_score > 8 AND nps_score <= 10 THEN 'promoters'
END AS nps_group
FROM user AS u LEFT JOIN (SELECT country, city, location_id
                       FROM location)
AS loc ON u.location_id = loc.location_id
LEFT JOIN (SELECT title, age_gr_id
        FROM age_segment)
AS a ON u.age_gr_id = a.age_gr_id
LEFT JOIN (SELECT title, tr_gr_id
        FROM traffic_segment)
AS tr ON u.tr_gr_id = tr.tr_gr_id
LEFT JOIN (SELECT title, lt_gr_id
        FROM lifetime_segment)
AS lt ON u.lt_gr_id = lt.lt_gr_id
"""

In [5]:
df = pd.read_sql(query,engine)
df.head(20)

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,old,45.0,1.0,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,promoters
1,A001WF,2344,old,53.0,0.0,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,promoters
2,A003Q7,467,old,57.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,promoters
3,A004TB,4190,old,44.0,1.0,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,promoters
4,A004XT,1163,old,24.0,0.0,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,promoters
5,A005O0,5501,old,42.0,1.0,ANDROID,SMARTPHONE,Россия,Омск,04 35-44,05 5-10,08 36+,6,detractors
6,A0061R,1236,old,45.0,0.0,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,06 10-15,08 36+,10,promoters
7,A009KS,313,new,35.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,04 35-44,13 45-50,05 7-12,10,promoters
8,A00AES,3238,old,36.0,1.0,ANDROID,SMARTPHONE,Россия,СанктПетербург,04 35-44,04 1-5,08 36+,10,promoters
9,A00F70,4479,old,54.0,1.0,ANDROID,SMARTPHONE,Россия,Волгоград,05 45-54,07 15-20,08 36+,9,promoters


In [6]:
df.to_csv('telecomm_csi_tableau.csv', index=False)

[Tableu дашборды](https://public.tableau.com/views/NPSresearch_17073679811040/Story1?:language=en-GB&:display_count=n&:origin=viz_share_link)

[Презентация](https://drive.google.com/file/d/1jIaPNvAC1n3q40lb_SVHIgsTgz1IWWMT/view?usp=sharing)